<a href="https://colab.research.google.com/github/Svetorus/Neuronets/blob/master/Neuronet_HW_5_%22%D0%90%D0%BB%D0%B8%D1%81%D0%B0_%D0%B2_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5_%D1%87%D1%83%D0%B4%D0%B5%D1%81%22_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Продолжение книги "Алиса в стране чудес" с помощью рекуррентной нейросети.

In [0]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 1 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию с 
multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/1
158773/158773 [==============================] - 17s 105us/step - loss: 2.3145
Генерация из посева: tle went o
tle went of the was in the was in the was in the was in the was in the was in the was in the was in the was in


### GRU
2 эпохи и 20 итераций

Итерация #: 0  
Epoch 1/2 
158773/158773 [==============================] - 18s 116us/step - loss: 2.3072  
Epoch 2/2  
158773/158773 [==============================] - 15s 96us/step - loss: 1.9152  
Генерация из посева: erpent! sa  
erpent! said the king the growe the growe the growe the growe the growe the growe the growe the growe the grow=  
Итерация #: 1  
please, the mouse a one of the was the was the was the was the was the was the was the was the was the was th=  
Итерация #: 2  
hief executions or a the caterpillar some of the realing alice was not a little soon as it was so the said to =  
Итерация #: 3  
fore, and the court on the court on the court on the court on the court on the court on the court on the court=  
Итерация #: 4  
and the brithtered to the mock turtle she had been the mock turtle she had been the mock turtle she had been t=  
Итерация #: 5  
between the dormouse soon as it was so she said to herself the dormouse soon as it was so she said to hersel=  
Итерация #: 6  
about it, said the king, and was so much and looking at the mouse, who was going on the words of the tried to=  
Итерация #: 7  
ing to a mouse seemed to her had a minute or two the court was a little shart so much and some time to alice w=  
Итерация #: 8  
over the very pinked herself to herself, and the mock turtle was a long as she was a long as she was a long as=  
Итерация #: 9  
i think-- theres no soot of the such a cried to the gryphon. there was a long to see it to herself, and the m=  
Итерация #: 10  
nd the executioner was so much of expecting to the gryphon, and the mock turtle seen the mock turtle seen the =  
Итерация #: 11  
nt put it one feniher the mouse of the same thing i had to see it was a little starts was the mock turtle went=  
Итерация #: 12  
ead:-- the other side of the work and the project gutenberg-tm electronic works in the tried to the gryphon as=  
Итерация #: 13  
jumping up and thing to the words for the end of the trees when the shorerul this agreement share of the same=  
Итерация #: 14  
en make out of the terms of the end of the treated the mock turtle see sharply and to great dish some terms of=  
Итерация #: 15  
e first witness was the door, so she began there was not a cance of a court was the baby at the white rabbit i=  
Итерация #: 16  
hind her. no doubt to think to the thing i had not gone at the mouse would be of the same thing i had not gone=  
Итерация #: 17  
t done, though she was a little bottle that she had not gone such a thing before, and she said to herself, and=  
Итерация #: 18  
g hand in a low voice, but she spoke. alice like the court, and then alice was reading that she was remarking =  
Итерация #: 19  
Epoch 1/2  
158773/158773 [==============================] - 17s 105us/step - loss: 0.9839  
Epoch 2/2  
158773/158773 [==============================] - 16s 102us/step - loss: 0.9778  
Генерация из посева: ings! on t  
ings! on their hands and began from the court, and was going to see it was a little bird and so many little sh

### LSTM
2 эпохи и 20 итераций

Итерация #: 0  
ing, and dont of the mare the mare the mare the mare the mare the mare the mare the mare the mare the mare the  
Итерация #: 1    
ct gutenberg-tm alice the mouse the mouse the mouse the mouse the mouse the mouse the mouse the mouse the mous  
Итерация #: 2    
he twinkling of the caterpillar what i think i she had not the caterpillar what i think i she had not the cate  
Итерация #: 3   
hing as i dont be a little soon the could not a conding to the court, and the could not a conding to the court  
Итерация #: 4    
he said to herself, and the mock turtle the roomsed and the book, and the mock turtle the roomsed and the bo  
Итерация #: 5    
but it dont be a little shart in the door as the door as the door as the door as the door as the door as the  
Итерация #: 6    
e particular and dont be no in the words to the thing to be a look of the works and the trees of the trees of  
Итерация #: 7    
t she was a little streat down and the some of the state with the door of the caterpillar. the mock turtle sai  
Итерация #: 8    
ueen, tossee, and went on the project gutenberg-tm electronic works to herself she went on the project gutenbe  
Итерация #: 9    
now what the first the project gutenberg-tm electronic work in a great hurry to herself, and the mock turtle s  
Итерация #: 10    
h a round her head through the dormouse in the little golden key and she could not the court, and she was a li  
Итерация #: 11    
their slates with the trees had a long the time, and the mock turtle said, and the mock turtle said, and the m  
Итерация #: 12    
loud. i should think of every thing is the white rabbit, said the caterpillar was the white rabbit, said the c  
Итерация #: 13    
t she added the dormouse sench, and and seeped to begin with, and she tried to beat of ever the hatter was go  
Итерация #: 14    
ed before, and the same thing a little thing the other side and some of the soldiers are a courter to ade way  
Итерация #: 15    
only walk a little birds and the dormouse say the duchess said to herself, and was going to listen to her fac  
Итерация #: 16    
pack rose with the gryphon, and the sound of the ground as the same thing a words hard is in the words do not  
Итерация #: 17    
f the mock turtle said, with a later on the bank--of course, then it would be one at her format that first, sa  
Итерация #: 18    
gain and a long tartle. the dormouse say there was not a serpent, the mock turtle said, and was going to liste  
Итерация #: 19    
Epoch 1/2  
158773/158773 [==============================] - 16s 100us/step - loss: 0.9380  
Epoch 2/2  
158773/158773 [==============================] - 16s 101us/step - loss: 0.9311  
Генерация из посева: u are old,  
u are old, found at the top of his head only say to into its a cheshire cat, she ducked head only say to into

### LSTM
10 эпох и 9 итераций

Итерация #: 0  
use did not all the tire, and the poor little thing in the piston the project gutenberg-tm electronic works in  
Итерация #: 1  
have a trial: you must comply or omplanent to her, she was a little while she was a little while she was a lit=  
Итерация #: 2  
white rabbit, who said in a low, with the dormouse shook its alice as he said in a low, with the dormouse sho=  
Итерация #: 3  
t yet, said alice to herself, as she was saying, and the shaichily as had fater with the terms of this eregard=  
Итерация #: 4  
ns hedgehog just now, only it ran away when it saw alice coming. there was not at all like the words did not s=  
Итерация #: 5  
owner of the other side of what? thought alice. im not a serpent! said the youth, as i mentioned dinah! she sa=  
Итерация #: 6  
e same tone, exactly as if nothing had happened. hellied segreder that she was clawing ob right- and broke off=  
Итерация #: 7  
if an individual project gutenberg-tm electronic work is discovered and reported to you as-is with no other w=  
Итерация #: 8  
Epoch 1/10  
158773/158773 [==============================] - 24s 149us/step - loss: 0.2955  
Epoch 2/10  
158773/158773 [==============================] - 22s 141us/step - loss: 0.2940  
Epoch 3/10  
158773/158773 [==============================] - 24s 148us/step - loss: 0.2931  
Epoch 4/10  
158773/158773 [==============================] - 26s 166us/step - loss: 0.2920  
Epoch 5/10  
158773/158773 [==============================] - 27s 168us/step - loss: 0.2921  
Epoch 6/10  
158773/158773 [==============================] - 26s 166us/step - loss: 0.2904  
Epoch 7/10  
158773/158773 [==============================] - 24s 154us/step - loss: 0.2898  
Epoch 8/10  
158773/158773 [==============================] - 23s 146us/step - loss: 0.2889  
Epoch 9/10  
158773/158773 [==============================] - 24s 150us/step - loss: 0.2886  
Epoch 10/10  
158773/158773 [==============================] - 22s 138us/step - loss: 0.2885  
Генерация из посева:  begun to   
begun to dream that she was now about it, you know, alice gently remarked, till tomorrow-- at this moment! th=  

### LSTM
5 эпох и 20 итераций

Итерация #: 0  
what was of the was soon the was soon the was soon the was soon the was soon the was soon the was soon the was=  
Итерация #: 1  
so there was a little began to see the mormation about the project gutenberg-tm electronic work in the court,  
Итерация #: 2  
 this a very story that she had to be a great hand of the court, said the dormouse said to herself as she said=  
Итерация #: 3  
continued, and the whole caught the white rabbit a little beautiful soup! said the dormouse she was a little b=  
Итерация #: 4  
on both sides to some of your sate in the same the end of the same the end of the same the end of the same th=  
Итерация #: 5  
whether youre a cantle our of the works out of the works out of the works out of the works out of the works o=  
Итерация #: 6  
mark on the book with her head in a low voice. alice was said to the tarts on the things the mock turtle seeme=  
Итерация #: 7  
at this moment the chilking, said the caterpillar. in a moment the chilking, said the caterpillar. in a mome=  
Итерация #: 8  
by his face in a thing as i think i can readly distributing or profuns of your for and dont say did not and d=  
Итерация #: 9  
must be read the mock turtle sighed she had not got with the pechain one had a minute or two, and the mouse w=  
Итерация #: 10  
attended to do before and down to her are seen what a great thing a little of it, said alice, in a long a cour=  
Итерация #: 11  
ldiers, who had been to alice as he spoke, and the mock turtle replied in an once of the mouse to alice as he =  
Итерация #: 12  
wn! said the mock turtle sighed she did not see you may gone, the mock turtle sighed she did not see you may g=  
Итерация #: 13  
can hardly took down a little while alice was those a little while alice was those a little while alice was t=  
Итерация #: 14  
n, was an offented to say tratwhing, i wonder what i should like to see a werm dinah, which he pat, said the c=  
Итерация #: 15  
now, dinah, the mock turtle replied, in a frecurses of the shore. and the mock turtle replied, in a frecurses =  
Итерация #: 16  
ject gutenberg-tm electronic works or aclordated indaced into herself, and she went on and satten, and she wen=  
Итерация #: 17  
ents like this wire, and she spoke. and when she had not a soon off, said the mock turtle into alices great cr=  
Итерация #: 18  
g about again. the rabbit suppressed and the dormouse thought that she was surprised at this to doon. there wa=  
Итерация #: 19  
Epoch 1/5  
158773/158773 [==============================] - 17s 106us/step - loss: 0.8241  
Epoch 2/5  
158773/158773 [==============================] - 16s 103us/step - loss: 0.8238  
Epoch 3/5  
158773/158773 [==============================] - 17s 109us/step - loss: 0.8230  
Epoch 4/5  
158773/158773 [==============================] - 17s 106us/step - loss: 0.8206  
Epoch 5/5  
158773/158773 [==============================] - 17s 108us/step - loss: 0.8184  
Генерация из посева:  here, tha  
here, thats all the rest of the project gutenberg-tm electronic work in the same thing a began down and lade.